In [324]:
import os

import numpy as np
import pandas as pd
from math import erfc, sqrt

from scipy import stats
import statsmodels.api as sm
import warnings

warnings.filterwarnings('ignore')
os.getcwd()

'/Volumes/GoogleDrive/My Drive/PhD/Asset Pricing/mehra_prescott'

In [325]:
data = pd.read_excel("./PCE growth data.xlsx", index_col="Year")

growth = data['Growth']
growth_lag1 = growth.shift(1).dropna()

growth = growth.iloc[1:]

data[['Growth']].describe().round(4)

,Growth
count,91.0000
mean,0.0173
std,0.0220
min,-0.0806
25%,0.0099
50%,0.0203
75%,0.0305
max,0.0737


## Fit an AR(1) model

In [327]:
# Fit model
Y = growth
X = growth_lag1

X = sm.add_constant(X)
model = sm.OLS(Y,X)
res = model.fit()

# Save parameters
u, rho = res.params  # Const + AR(1) param
sigma = np.sqrt(res.scale)
reg_vars = [u, sigma, rho]

print('σ = ',res.scale**(0.5))
res.params

σ =  0.018699544784718813


const     0.009307
Growth    0.479500
dtype: float64

In [356]:
# Compute unconditional moments
ar_mean = u / (1-rho) + 1
ar_std = np.sqrt(sigma**2 / (1-rho**2))
ar_rho = rho #rho * ar_std**2

ar_vars = [ar_mean, ar_std, ar_rho]

# Summarize results
summ = pd.DataFrame([reg_vars, ar_vars],
             columns=['mean', 'std', 'rho'],
             index=['regression', 'ar_moments'])
summ

,mean,std,rho
regression,0.009307,0.018700,0.4795
ar_moments,1.017882,0.021309,0.4795


## Calibrate markov chain

In [357]:
Z, P = rouwenhorst(n=2, mu=ar_mean, sigma=ar_std, rho=rho)  # state-vector and TPM

Z, P

(array([0.99657272, 1.03919074]),
 array([[0.73975015, 0.26024985],
        [0.26024985, 0.73975015]]))

In [358]:
# Compute unconditional probabilities (ergodic distribution):
eig_val, eig_vec = np.linalg.eig(P.T)
eig_vec = eig_vec[:, np.argmax(eig_val)]
pi_bar = eig_vec / eig_vec.sum()

# Compute unconditional probabilities (ergodic distribution) with brute force:
pi_brute = np.linalg.matrix_power(P, 1000)[0]

print("Check if stationary dist is same for both methods: ",
      (pi_bar.round(10) == pi_brute.round(10)).all(), "\n","-"*40)
pi_bar

Check if stationary dist is same for both methods:  True 
 ----------------------------------------


array([0.5, 0.5])

### Get moments of markov chain

In [359]:
m_mean = pi_bar @ Z
m_std = np.sqrt(pi_bar @ Z**2 - (pi_bar @ Z)**2)
m_rho = np.trace(P) - 1

m_vars = [m_mean, m_std, m_rho]

summ = summ.append(pd.Series(m_vars, index=summ.columns, name='mc2'))
summ

,mean,std,rho
regression,0.009307,0.018700,0.4795
ar_moments,1.017882,0.021309,0.4795
mc2,1.017882,0.021309,0.4795


In [362]:
Z, P = rouwenhorst(n=10, mu=ar_mean, sigma=ar_std, rho=rho)  # state-vector and TPM
# Compute unconditional probabilities (ergodic distribution):
eig_val, eig_vec = np.linalg.eig(P.T)
eig_vec = eig_vec[:, np.argmax(eig_val)]
pi_bar = eig_vec / eig_vec.sum()

m_mean = pi_bar @ Z
m_std = np.sqrt(pi_bar @ Z**2 - (pi_bar @ Z)**2)
m_rho = np.trace(P) - 1

m_vars = [m_mean, m_std, m_rho]

summ = summ.append(pd.Series(m_vars, index=summ.columns, name='mc10'))
summ

,mean,std,rho
regression,0.009307,0.018700,0.479500
ar_moments,1.017882,0.021309,0.479500
mc2,1.017882,0.021309,0.479500
mc10,1.017882,0.021309,0.919996
mc10,1.017882,0.021309,-0.786667


In [355]:
def rouwenhorst(n, mu, sigma, rho):
    r"""
    Takes as inputs n, p, q, psi. It will then construct a markov chain
    that estimates an AR(1)
    
    Uses mu, sigma and rho directly from AR regression.

    Parameters
    ----------
    n : int
        The number of points to approximate the distribution
    mu : float
        The value :math:`\bar{y}` in the process.  Note that the mean of this
        AR(1) process, :math:`y`, is simply :math:`\bar{y}/(1 - \rho)`
    sigma : float
        The value of the standard deviation of the :math:`\varepsilon` process
    rho : float
        By default this will be 0, but if you are approximating an AR(1)
        process then this is the autocorrelation across periods
    """

    # Get the standard deviation of y
    sigmaz = sigma #sqrt(sigma**2 / (1 - rho**2))

    p = (1 + rho) / 2
    
    PI = row_build_mat(n, p, p)  # Build transition matrix
    
    fi = np.sqrt(n - 1) * sigmaz
    Z = np.linspace(-fi, fi, n)
    Z = Z + mu

    return Z, PI


def row_build_mat(n, p, q):
    """
    This method uses the values of p and q to build the transition
    matrix for the rouwenhorst method
    """

    if n == 2:
        theta = np.array([[p, 1 - p], [1 - q, q]])

    elif n > 2:
        p1 = np.zeros((n, n))
        p2 = np.zeros((n, n))
        p3 = np.zeros((n, n))
        p4 = np.zeros((n, n))

        new_mat = row_build_mat(n - 1, p, q)

        p1[:n - 1, :n - 1] = p * new_mat
        p2[:n - 1, 1:] = (1 - p) * new_mat
        p3[1:, :-1] = (1 - q) * new_mat
        p4[1:, 1:] = q * new_mat

        theta = p1 + p2 + p3 + p4
        theta[1:n - 1, :] = theta[1:n - 1, :] / 2

    else:
        raise ValueError("The number of states must be positive " +
                         "and greater than or equal to 2")

    return theta

In [198]:
mu = 0.0183
sigma = 0.0357
rho = -0.14


rouwenhorst(n=2, ybar=mu, sigma=sigma, rho=rho)

(array([[0.43, 0.57],
        [0.57, 0.43]]),
 array([-0.02000246,  0.05210772]))

In [43]:
mu + sigma

0.054000000000000006

In [44]:
mu - sigma

-0.017400000000000002